In [18]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [78]:
# URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
path="./data/heart.csv"
dataframe = pd.read_csv(path)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [63]:
for i in dataframe.columns:
    if dataframe[i].dtype==np.int64:
        dataframe[i]=dataframe[i].astype(np.float64)

In [64]:
for i in dataframe.columns:
    print(dataframe[i].dtype)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
float64


In [65]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


In [66]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [67]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [68]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['age'])
    print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([62. 48. 57. 61. 50.], shape=(5,), dtype=float32)
A batch of targets: tf.Tensor([1. 0. 0. 0. 0.], shape=(5,), dtype=float32)


In [69]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [70]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [71]:
age = feature_column.numeric_column("age")
demo(age)

[[57.]
 [63.]
 [64.]
 [46.]
 [41.]]


In [84]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [73]:
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [74]:
# Notice the input to the embedding column is the categorical column
# we previously created
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[ 0.20886762  0.01882742 -0.17041926  0.05419277  0.18896168 -0.48949665
   0.22594008 -0.63486624]
 [-0.3312263   0.3738114   0.0885167   0.2751385  -0.21731652 -0.22174965
  -0.00391643 -0.06544495]
 [-0.3312263   0.3738114   0.0885167   0.2751385  -0.21731652 -0.22174965
  -0.00391643 -0.06544495]
 [-0.3312263   0.3738114   0.0885167   0.2751385  -0.21731652 -0.22174965
  -0.00391643 -0.06544495]
 [-0.15010685 -0.13880129 -0.6689409   0.2238306   0.38441166  0.032855
  -0.0273873   0.5244997 ]]


In [75]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [76]:
example_batch

{'age': <tf.Tensor: id=1162, shape=(5,), dtype=float32, numpy=array([57., 63., 64., 46., 41.], dtype=float32)>,
 'sex': <tf.Tensor: id=1170, shape=(5,), dtype=float32, numpy=array([1., 0., 1., 1., 0.], dtype=float32)>,
 'cp': <tf.Tensor: id=1165, shape=(5,), dtype=float32, numpy=array([4., 4., 4., 4., 3.], dtype=float32)>,
 'trestbps': <tf.Tensor: id=1174, shape=(5,), dtype=float32, numpy=array([110., 150., 128., 120., 112.], dtype=float32)>,
 'chol': <tf.Tensor: id=1164, shape=(5,), dtype=float32, numpy=array([201., 407., 263., 249., 268.], dtype=float32)>,
 'fbs': <tf.Tensor: id=1167, shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 'restecg': <tf.Tensor: id=1169, shape=(5,), dtype=float32, numpy=array([0., 2., 0., 2., 2.], dtype=float32)>,
 'thalach': <tf.Tensor: id=1173, shape=(5,), dtype=float32, numpy=array([126., 154., 105., 144., 172.], dtype=float32)>,
 'exang': <tf.Tensor: id=1166, shape=(5,), dtype=float32, numpy=array([1., 0., 1., 0., 1.], dtyp

In [79]:
crossed_feature

CrossedColumn(keys=(BucketizedColumn(source_column=NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(18, 25, 30, 35, 40, 45, 50, 55, 60, 65)), VocabularyListCategoricalColumn(key='thal', vocabulary_list=('fixed', 'normal', 'reversible'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), hash_bucket_size=1000, hash_key=None)

In [85]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

SystemError: <built-in function TFE_Py_FastPathExecute> returned a result with an error set

In [83]:
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
layers.DenseFeatures(feature_column.indicator_column(crossed_feature))(example_batch)

SystemError: <built-in function TFE_Py_FastPathExecute> returned a result with an error set

In [50]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
      feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [51]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [52]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [53]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/5
7/7 [==============================] - 2s 221ms/step - loss: 1.0084 - accuracy: 0.5596 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
7/7 [==============================] - 0s 16ms/step - loss: 1.1228 - accuracy: 0.6684 - val_loss: 0.4323 - val_accuracy: 0.8367
Epoch 3/5
7/7 [==============================] - 0s 16ms/step - loss: 0.7983 - accuracy: 0.7358 - val_loss: 0.4335 - val_accuracy: 0.7551
Epoch 4/5
7/7 [==============================] - 0s 17ms/step - loss: 0.4923 - accuracy: 0.7720 - val_loss: 0.4247 - val_accuracy: 0.7551
Epoch 5/5
7/7 [==============================] - 0s 16ms/step - loss: 0.5257 - accuracy: 0.7306 - val_loss: 0.4234 - val_accuracy: 0.8163


In [54]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 6ms/step - loss: 0.4317 - accuracy: 0.8197
Accuracy 0.8196721
